In [ ]:
%load_ext autoreload
%autoreload 2

In [39]:
schema = 'sales'
catalog = 'antonio_junior_raw'
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {catalog}.sales"
spark.sql(create_schema_sql)

""


In [8]:
password = dbutils.secrets.get(scope="antonio_junior_adw", key="pswd_mssql")
host = dbutils.secrets.get(scope="antonio_junior_adw", key="ip_mssql")
port = dbutils.secrets.get(scope="antonio_junior_adw", key="port_mssql")

jdbcUrl = f"jdbc:sqlserver://{host}:{port};databaseName=AdventureWorks;encrypt=true;trustServerCertificate=true;"

connectionProperties = {
    "user": "SA",  # Replace "SA" with the appropriate username if needed
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}



In [9]:
query = """
SELECT TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_SCHEMA = 'sales'
"""

In [30]:
try:
    tables_list = spark.read.jdbc(url=jdbcUrl, table=f"({query}) AS tmp", properties=connectionProperties)
    print("List of tables in the 'sales' schema retrieved successfully.")
except Exception as e:
    print(f"Failed to retrieve tables: {e}")


List of tables in the 'sales' schema retrieved successfully.


In [43]:
for table in tables_list.toLocalIterator():
    table_name = table.TABLE_NAME
    print(table_name)
    df = spark.read.jdbc(url=jdbcUrl, table=f'{schema}.{table_name}', properties=connectionProperties)
    df.write \
      .format("delta") \
      .mode("overwrite") \
      .option("overwriteSchema", "true") \
      .saveAsTable(f"{catalog}.{schema}.{table_name}")



SalesTaxRate
PersonCreditCard
SalesTerritory
vIndividualCustomer
vPersonDemographics
vSalesPerson
SalesTerritoryHistory
vSalesPersonSalesByFiscalYears
vStoreWithDemographics
vStoreWithContacts
vStoreWithAddresses
ShoppingCartItem
SpecialOffer
SpecialOfferProduct
Store
CountryRegionCurrency
CreditCard
Currency
CurrencyRate
Customer
SalesOrderDetail
SalesOrderHeader
SalesOrderHeaderSalesReason
SalesPerson
SalesPersonQuotaHistory
SalesReason


In [13]:
table = 'SalesTaxRate'




""


In [21]:
spark.sql(f"SHOW SCHEMAS IN {catalog}").show()


+------------------+
|      databaseName|
+------------------+
|information_schema|
|             sales|
+------------------+

